In [3]:
// Configure the connection to the data lake
var adlsPath = "abfss://wwi@azsynapseaiddl.dfs.core.windows.net";

In [4]:
// Import the sales data from the data lake
var sales = spark.Read().Option("header", "true").Option("inferSchema", "true").Option("sep", "|").Csv($"{adlsPath}/factsale-csv/2012/Q4");
sales.Show(5);
sales.PrintSchema();

In [6]:
using System;
using System.Collections.Generic;

// Set up a JDBC connection to Azure Synapse Analytics
var jdbcUsername = "asa.sql.staging@az-synapse-aid-lab-ws";
var jdbcPassword = "Temp1234";
var jdbcConnectionString = $"jdbc:sqlserver://az-synapse-aid-lab-ws.sql.azuresynapse.net:1433;database=sqlpool01;";

// Set target table name
var tableName = $"wwi_staging.Sale_CSharp";

// Write the retrieved sales data into a staging table in Azure Synapse Analytics.
sales
    .Limit(1500)
    .Write()
    .Format("jdbc")
    .Options(
        new Dictionary<string, string>
        {
            {"url", jdbcConnectionString},
            {"dbtable", tableName},
            {"user", jdbcUsername},
            {"password", jdbcPassword},
            {"encrypt", "false"},
            {"trustServerCertificate", "true"},
            {"createTableColumnTypes", "Description VARCHAR(100), Package VARCHAR(50)"} // Override the column types for string values. They are assigned nvarchar(max) by default, and that is not compatible with columnstore index.
        }
    )
    .Mode("append")
    .Save();